In [22]:
import pandas as pd
from lib.utils import get_id_nums, clean_row, remove_incorrect_codes

In [7]:
pd.set_option('display.max_rows', 100)

In [4]:
ct = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/final/countries.csv')

## Country Work

In [23]:
countries_cols = ['name', 'alpha-2', 'region', 'sub-region']
countries = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/all.csv', usecols=countries_cols, keep_default_na=False)

extra_codes = pd.DataFrame({'name': ['Czechia', 'Netherland Antilles', 'Germany', 'European Union'], 'alpha-2': ['XC', 'AN', 'DD', 'EU'], 'region': ['Europe', 'Americas', 'Europe', 'Europe'], 'sub-region': ['Eastern Europe', 'Latin America and the Caribbean', 'Western Europe', 'Western Europe']})

extra_codes
countries = pd.concat([countries, extra_codes], ignore_index=True, keys=['alpha-2', 'name'])

# Change countries index column to be alpha-2 values and rename to id
countries.set_index('alpha-2', inplace=True)
countries.index.name = 'id'
countries.sort_index(inplace=True)

alpha_2_set = set(countries.index)

country_dict = {x[1].upper(): x[0] for x in countries.itertuples()}
country_dict['TAIWAN'] = 'TW'
country_dict['SOUTH KOREA'] = 'KR'
country_dict['SHANGHAI CN'] = 'CN'
country_dict['SHANGHAI'] = 'CN'
country_dict['SHANGHAI .'] = 'CN'
country_dict['HONG KONG .'] = 'CN'
country_dict['TAIPEI .'] = 'TW'
country_dict['USA'] = 'US'
country_dict['U.S.A.'] = 'US'

In [ ]:
countries

In [6]:
countries_file_path = '/Users/jesseputnam/cs-learning/skillstorm/project01/data/final/countries.csv'
countries.to_csv(countries_file_path)

# Shipper work

In [40]:
shipper_sample = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/sample/shipper_2018.csv')
shipper_sample['shipper_party_name'].fillna('N/A', inplace=True)

In [41]:
shipper_sample_clean = shipper_sample.apply(lambda row: clean_row(row, 'shipper_party', alpha_2_set, country_dict), axis=1)
shipper_sample_clean = shipper_sample_clean[['identifier', 'shipper_party_name', 'country_code']]
shipper_sample_ids = get_id_nums(shipper_sample_clean['shipper_party_name'])
shipper_sample_clean['shipper_id'] = shipper_sample_clean['shipper_party_name'].map(shipper_sample_ids)

In [ ]:
shipper_sample_clean

In [ ]:
shipper_sample_clean.info()

In [82]:
shipper_keep_cols = ['identifier', 'shipper_party_name', 'shipper_party_address_1', 'shipper_party_address_2', 'shipper_party_address_3', 'shipper_party_address_4', 'country_code']

shipper_2018_0 = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/2018/shipper_2018_part_0.csv', index_col=False, usecols=shipper_keep_cols)

# Dealing with missing names
shipper_2018_0['shipper_party_name'].fillna('N/A', inplace=True)

In [ ]:
shipper_2018_0[shipper_2018_0['shipper_party_name'] == "MR. AND/OR MRS."]

In [4]:
# shipper_test = shipper_2018_0.apply(lambda row: clean_shipper(row, alpha_2_set, country_dict), axis=1)

In [ ]:
# shipper_test['shipper_party_name'].value_counts().sort_index().head(90)

In [6]:
# shipper_test = shipper_test[['identifier', 'shipper_party_name', 'country_code']]

In [ ]:
# shipper_test.info(show_counts=True)

### Dealing with Names

In [12]:
# shipper_sample_cleaned = shipper_sample.apply(lambda row: clean_shipper(row, alpha_2_set, country_dict), axis=1)
# shipper_sample_cleaned = shipper_sample_cleaned[['identifier', 'shipper_party_name', 'country_code']]

## Mapping names to id numbers

In [ ]:
# shipper_id_dict = {}
# shipper_id_count = 0

# for name in shipper_sample_cleaned['shipper_party_name'].unique():
#     shipper_id_dict[name] = shipper_id_count
#     shipper_id_count += 1

# shipper_sample_cleaned['shipper_id'] = shipper_sample_cleaned['shipper_party_name'].map(shipper_id_dict)
# shipper_sample_cleaned


### Combine shipper csv's to one file

In [ ]:
# shipper_raw_path = '/Users/jesseputnam/cs-learning/skillstorm/project01/data/final_raw/shipper.csv'

# shipper_0 = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/2018/shipper_2018_part_0.csv', usecols=shipper_keep_cols)
# shipper_1 = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/2018/shipper_2018_part_1.csv', usecols=shipper_keep_cols)
# shipper_2 = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/2019/shipper_2019_part_0.csv', usecols=shipper_keep_cols)
# shipper_3 = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/2019/shipper_2019_part_1.csv', usecols=shipper_keep_cols)
# shipper_4 = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/2020/shipper_2020_part_0.csv', usecols=shipper_keep_cols)
# shipper_5 = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/2020/shipper_2020_part_1.csv', usecols=shipper_keep_cols)

In [ ]:
consignee_sample = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/sample/consignee_2018.csv')
consignee_sample['consignee_name'].fillna('N/A', inplace=True)

In [46]:
consignee_sample_clean = consignee_sample.apply(lambda row: clean_row(row, 'consignee', alpha_2_set, country_dict), axis=1)
consignee_sample_clean = consignee_sample_clean[['identifier', "consignee_name", 'country_code']]

In [48]:
consignee_sample_ids = get_id_nums(consignee_sample_clean['consignee_name'])
consignee_sample_clean['consignee_id'] = consignee_sample_clean['consignee_name'].map(consignee_sample_ids)

In [12]:
consignee_cols = ['identifier', 'consignee_name', 'consignee_address_1', 'consignee_address_2', 'consignee_address_3', 'consignee_address_4', 'country_code']

consignee_full = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/2018/consignee_2018_part_0.csv', usecols=consignee_cols)
consignee_full['consignee_name'].fillna('N/A', inplace=True)

In [ ]:
consignee_full['consignee_address_3'].value_counts()

In [ ]:
consignee_full.info(show_counts=True)

- Remove erroneous country codes

In [6]:
# consignee_full_clean = consignee_full.apply(lambda row: clean_consignee(row, alpha_2_set), axis=1)

In [ ]:
consignee_full['consignee_name'].value_counts(sort=False).sort_index().head(50)

In [ ]:
# consignee_full_clean['consignee_name'].value_counts(sort=False).sort_index().head(50)

In [9]:
# consignee_id_dict = get_id_nums(consignee_full_clean['consignee_name'])
# consignee_full_clean['consignee_id'] = consignee_full_clean['consignee_name'].map(consignee_id_dict)

In [ ]:
# consignee_full_clean['consignee_id'].value_counts()

## Header Work

In [80]:
header_keep_cols = ['vessel_name', 'vessel_country_code', 'carrier_code', 'identifier', 'estimated_arrival_date', 'actual_arrival_date']

header_2018_0 = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/2018/header_2018_part_0.csv', parse_dates=['estimated_arrival_date', 'actual_arrival_date'], usecols=header_keep_cols)

In [70]:
header_2018_0['vessel_country_code'] = header_2018_0['vessel_country_code'].apply(lambda code: remove_incorrect_codes(code, alpha_2_set))

In [81]:
header_sample = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/sample/header_2018.csv', usecols=header_keep_cols)

In [ ]:
consignee_sample_clean.head()

In [ ]:
shipper_sample_clean.head()

In [106]:
header_full_table = header_sample.merge(shipper_sample_clean[['identifier', 'shipper_id']], on='identifier', how='inner')
header_full_table = header_full_table.merge(consignee_sample_clean[['identifier', 'consignee_id']], on='identifier', how='inner')

In [ ]:
header_full_table.head(10)

In [ ]:
header_sample.info()